In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cupy as cp

In [ ]:
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, kernel="linear", gamma=0.1):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.kernel_type = kernel
        self.gamma = gamma
        self.w = None
        self.b = None

    def kernel(self, X1, X2):
        """Compute kernel function using CuPy for GPU acceleration."""
        if self.kernel_type == "linear":
            return cp.dot(X1, X2.T)
        elif self.kernel_type == "rbf":
            X1 = X1[:, cp.newaxis]
            return cp.exp(-self.gamma * cp.linalg.norm(X1 - X2, axis=2) ** 2)
        else:
            raise ValueError("Unsupported kernel")

    def fit(self, X, y):
        """Train SVM model using GPU."""
        X = cp.asarray(X)  # Move data to GPU
        y = cp.asarray(y)
        y = cp.where(y <= 0, -1, 1)  # Convert labels to {-1, 1}
        n_samples, n_features = X.shape

        if self.kernel_type == "linear":
            self.w = cp.zeros(n_features)
        else:
            self.alpha = cp.zeros(n_samples)  # Dual coefficients
        self.b = 0

        K = self.kernel(X, X)  # Compute kernel matrix on GPU

        for _ in range(self.n_iters):
            for idx in range(n_samples):
                condition = y[idx] * (cp.dot(X[idx], self.w) + self.b) >= 1 if self.kernel_type == "linear" else \
                            y[idx] * cp.sum(self.alpha * y * K[idx]) + self.b >= 1

                if condition:
                    if self.kernel_type == "linear":
                        self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    if self.kernel_type == "linear":
                        self.w -= self.lr * (2 * self.lambda_param * self.w - cp.dot(X[idx], y[idx]))
                        self.b -= self.lr * y[idx]
                    else:
                        self.alpha[idx] += self.lr  # Adjust alpha for kernelized SVM
                        self.b -= self.lr * y[idx]

        self.X_train = X  # Store training data
        self.y_train = y

    def predict(self, X):
        """Make predictions using GPU acceleration."""
        X = cp.asarray(X)  # Move test data to GPU

        if self.kernel_type == "linear":
            return cp.sign(cp.dot(X, self.w) + self.b)
        else:
            K = self.kernel(X, self.X_train)
            return cp.sign(cp.sum(self.alpha * self.y_train * K, axis=1) + self.b)



In [ ]:
def plot_svm(X, y, model):
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm')

    # Plot decision boundary
    x0_1 = np.min(X[:, 0])
    x0_2 = np.max(X[:, 0])
    x1_1 = (-model.w[0] * x0_1 - model.b) / model.w[1]
    x1_2 = (-model.w[0] * x0_2 - model.b) / model.w[1]

    plt.plot([x0_1, x0_2], [x1_1, x1_2], 'k')
    plt.show()

In [ ]:
# from math import nan
df = pd.read_csv("/content/drive/MyDrive/ML/Dataset/train.csv")
# df.head()
# print(df.dtypes)
# (df==nan).sum()

In [ ]:
# df = df.iloc[:2000]

In [ ]:
Y = df["label"].to_numpy()
X = df.drop(columns=["label"]).to_numpy()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)


Shape of X_train: (37800, 784)
Shape of Y_train: (37800,)


In [1]:
# Train the model
svm = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=10)
svm.fit(X_train, Y_train)

# Predict on training data
y_pred = svm.predict(X_test)

In [ ]:
# Move y_pred and Y_test to CPU before computing accuracy
accuracy = np.mean(Y_test == y_pred.get())  # .get() converts CuPy array to NumPy
print("SVM Accuracy:", accuracy)


SVM Accuracy: 0.11214285714285714


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Train SVM with RBF kernel
svm = SVC(kernel="rbf", C=1.0, gamma="scale")  # C controls regularization
svm.fit(X_train, Y_train)

# Evaluate model
y_pred = svm.predict(X_test)
accuracy = np.mean(Y_test == y_pred)

print("SVM Accuracy:", accuracy)


SVM Accuracy: 0.9759523809523809
